In [1]:
import serial
import time
import pyvisa 

# You will have to change this to whatever COM port the pico is assigned when
# you plug it in.
# On Windows you can open device manager and look at the 'Ports (COM & LPT)' dropdown
# the pico will show up as 'USB Serial Device'
PICO_PORT = 'COM3'

MHZ = 1000000

SAVE_IMAGES = False

In [2]:
# helper for sending serial commands
# expects a string with the command (they dont have to be escaped with \r or \n at the end)
# if echo is set to false, that means not to worry about the response from the
# pico it signifigantly reduces communication time when sending many
# instructions, but you lose out on the debugging info from the pico
def send(command: str, echo = True) -> str:
    # pico is expecting a newline to end every command
    if command[-1] != '\n':
        command += '\n'

    resp = ''
    conn = None
    try:
        conn = serial.Serial(PICO_PORT, baudrate = 152000, timeout = 0.1)
        conn.write(command.encode())
        if echo:
            resp = conn.readlines()
            resp = "".join([s.decode() for s in resp])

    except Exception as e:
        print("Encountered Error: ", e)

    finally:
        conn.close()

    return resp


## Test Serial Communication with the Pico

In [3]:
assert send('reset')    == 'ok\n'
assert send('status')   == '0\n'
assert send('version')  == '0.2.1\n'
print('Serial Communication Successful')

Serial Communication Successful


## Test Register Readback

In [4]:
# Helper for reading register values and putting them in a dictionary
# takes in the frequency of the reference clock, assuming default of 125 MHz
def readregs(ref_clk = 125 * MHZ) -> dict:
    ad9959 = {}

    regs = send('readregs')
    regs = regs.split('\n')
    # strip out register labels
    regs = [''.join(r.split()[1:]) for r in regs]

    # convert from hex to decimal
    for i, reg in enumerate(regs):
        try:
            regs[i] = int(reg, 16)
        except ValueError:
            pass

    # mask and shift to pull out PLL Multiplier from fr1
    ad9959['pll_mult'] = (regs[1] & 0x7c0000) >> 18

    # will need system clock to find the frequencies from the tuning words
    sys_clk = ref_clk * ad9959['pll_mult']

    for i in range(4):
        ad9959[i] = {}

        ftw = regs[5 + 9 * i]
        ad9959[i]['freq'] = ftw / 2**32 * sys_clk

        pow = regs[6 + 9 * i]
        ad9959[i]['phase'] = pow * 360 / 2**14

        acr = regs[7 + 9 * i]
        if acr & 0x001000:
            ad9959[i]['amp'] = (acr & 0x0003ff) / 1023
        else:
            ad9959[i]['amp'] = 1
    
    return ad9959


readregs()


{'pll_mult': 4,
 0: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 1: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 2: {'freq': 0.0, 'phase': 0.0, 'amp': 1},
 3: {'freq': 0.0, 'phase': 0.0, 'amp': 1}}

In [5]:
assert send('reset') == 'ok\n', 'Could not run "reset" command'
ad9959 = readregs()
for i in range(4):
    assert ad9959[i]['freq'] == 0
    assert ad9959[i]['phase'] == 0
    assert ad9959[i]['amp'] == 1

send('setfreq 0 100000000')
send('setphase 1 270')
send('setamp 2 0.5')

ad9959 = readregs()

assert abs(ad9959[0]['freq'] - 100 * MHZ) < 1
assert abs(ad9959[1]['phase'] - 270) < 1
assert abs(ad9959[2]['amp'] - 0.5) < 0.01

print('Register Readback Successful')

Register Readback Successful


In [52]:
class KeysightScope:
    """
    Helper class leveraging pyvisa for a Keysight Oscilloscope 
    """
    def __init__(self, addr='USB?*::INSTR',
                 timeout=1, termination='\n'):
        rm = pyvisa.ResourceManager()
        devs = rm.list_resources(addr)
        assert len(devs), "pyvisa didn't find any connected devices matching " + addr
        self.dev = rm.open_resource(devs[0])
        self.dev.timeout = 15_000 * timeout
        self.dev.read_termination = termination
        self.idn = self.dev.query('*IDN?')
        self.read = self.dev.read
        self.write = self.dev.write
        self.query = self.dev.query

    def _get_screenshot(self, verbose=False):
        if verbose:
            print('Acquiring screen image...')
        return(self.dev.query_binary_values(':DISPlay:DATA? PNG, COLor', datatype='s'))

    def save_screenshot(self, filepath, verbose=False):
        if verbose:
            print('Saving screen image...')
        result = self._get_screenshot()
        with open(f'{filepath}', 'wb+') as ofile:
            ofile.write(bytes(result))

    def set_time_delay(self, time):
        self.write(f':TIMebase:DELay {time}')

    def set_time_scale(self, time):
        self.write(f':TIMebase:SCALe {time}')

my_instrument = KeysightScope()


if SAVE_IMAGES == True:
    my_instrument.save_screenshot('startup.png')


## JP Testing

### Different Freqs


In [7]:
## source
freq0 = 90 * MHZ # source freq > ref freq
phase0 = 0
amp0 = 1

## reference
freq1 = 85 * MHZ
phase1 = 270
amp1 = 1

send(f'setfreq 0 {freq0}')
send(f'setphase 0 {phase0}')
send(f'setamp 0 {amp0}')

send(f'setfreq 1 {freq1}')
send(f'setphase 1 {phase1}')
send(f'setamp 1 {amp1}')

'ok\n'

### Different Phases


In [8]:
# send(f"""
#      abort
#      mode 0 1
#      setchannels 1
#      """)
## source
freq0 = 100 * MHZ # source freq > ref freq
phase0 = 0
amp0 = 1

## reference
freq1 = 100 * MHZ
phase1 = 270
amp1 = 1

send(f'setfreq 0 {freq0}')
send(f'setphase 0 {phase0}')
send(f'setamp 0 {amp0}')

send(f'setfreq 1 {freq1}')
send(f'setphase 1 {phase1}')
send(f'setamp 1 {amp1}')

'ok\n'

### Odd problem

In [9]:
# send('reset')
# send('mode 0 1')
# send('setchannels 1')

# print(send('set 0 0 100_000_000 1 0 500_000'))
# print(send('set 4 1')) # this works fine

# send('reset')
# send('mode 0 1')
# send('setchannels 1')

# print(send('set 4 1'))  # this fails
# print(send('set 0 0 100_000_000 1 0 500_000'))

### Set Viewing Params


In [10]:
# my_instrument.query('*LRN?')
# my_instrument.query('TIMebase:MODE?')
my_instrument.write(":CHANnel1:DISPlay 1")
my_instrument.write(':ChANnel1:SCALe +250E-03')
my_instrument.write(':ChANnel1:OFFSet -450E-03')

my_instrument.write(":CHANnel2:DISPlay 1")
my_instrument.write(':ChANnel2:SCALe +1.00E+00')
my_instrument.write(':ChANnel2:OFFSet 275E-03')

my_instrument.write(":CHANnel3:DISPlay 1")
my_instrument.write(':ChANnel3:SCALe +1.00E+00')
my_instrument.write(':ChANnel3:OFFSet +2.0E+00')

my_instrument.write(":CHANnel4:DISPlay 1")
my_instrument.write(':ChANnel4:SCALe +1.00E+00')
my_instrument.write(':ChANnel4:OFFSet +2.0E+00')

## ensure Source 1 and Source 2 are the outputs of the PFD on your setup
my_instrument.write(':FUNCtion:OPERation SUBTract')

30

## Test Mode 0: Single Stepping Table Mode

Program a 2000 step table that single steps from 10 MHz to 100 MHz over the course of 2 seconds.
The resulting sweep can easily be seen with a spectrum analyzer. 
It is then automatically executed and checks that all 2000 triggers were processed successfully.

In [11]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 0: Single Stepping"')
send('reset')
my_instrument.write(':TIMebase:DELay 20.0E-06')
my_instrument.write(':TIMebase:SCALe 10.0E-09')
my_instrument.write(":CHANnel2:DISPlay 0")

21

In [12]:
startPoint = 10 * MHZ
endPoint = 100 * MHZ
totalTime = 2 # sec

spacing = 1000 * 10**(-6) # us
steps = round(totalTime / spacing)
delta = (endPoint - startPoint) / steps

send('debug off')
send('mode 0 1')
send('setchannels 1')

for i in range(steps):
    send(f'set 0 {i} {startPoint + delta * i} 1 0 {spacing * 10**9 / 8}', echo=False)
assert send(f'set 4 {i + 1}') == "ok\n"

print("Table Programmed, Executing")
assert send('start') == 'ok\n', 'Buffered Execution did not start correctly'
time.sleep(totalTime)
assert send('numtriggers') == f'{steps}\n', 'Wrong number of triggers processed'
print('Table Executed successfully')
if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode0-test.png')


Table Programmed, Executing
Table Executed successfully


### Test Non-Volatile Storage
Run this test after the previous one to test storing and retrieving table instructions in non-volatile memory that will survive a power cycle.

In [13]:
send('save')

# # destory current table
time.sleep(2)
for i in range(steps):
    send(f'set 0 {i} 0 0 0 0', echo=False)

# load and run table
send('load')
time.sleep(1)
send('mode 0 1')
send('setchannels 1')
send('start')

time.sleep(2)
assert send('numtriggers') == '2000\n', 'Something went wrong'
print('Table run from non-volatile memory successfully')

if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode0-recall-test.png')

Table run from non-volatile memory successfully


## Mode 1: Amplitude Sweep

### Pico Start

In [14]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 1"')
my_instrument.write(':TIMebase:DELay 35.0E-06')
my_instrument.write(':TIMebase:SCALe 10.0E-06')
my_instrument.write(":CHANnel2:DISPlay 1")
my_instrument.write(":TRIGger:EDGE:SOURce CHANnel2")

send('reset')

'ok\n'

In [15]:
send('debug off')
send(f"""mode 1 1
setchannels 1
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
set 0 0 1.0 0.0 0.001 1 2000
set 0 1 0.0 0.5 0.001 1 2000
set 0 2 0.5 1.0 0.001 1 2000
set 0 3 1.0 0.0 0.001 1 2000
set 0 4 0.0 1.0 0.001 1 2000
set 0 5 1.0 0.5 0.001 1 2000
set 0 6 0.5 0.0 0.001 1 2000
set 0 7 0.0 1.0 0.001 1 2000
set 4 8
start
""")

assert send('numtriggers') == '8\n'
print('Success')
if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode1-picostart-test.png')

Success


### HWStart

In [16]:
send('debug off')
send("""abort
mode 1 1
setchannels 1
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
set 0 0 1.0 0.0 0.001 1 2000
set 0 1 0.0 0.5 0.001 1 2000
set 0 2 0.5 1.0 0.001 1 2000
set 0 3 1.0 0.0 0.001 1 2000
set 0 4 0.0 1.0 0.001 1 2000
set 0 5 1.0 0.5 0.001 1 2000
set 0 6 0.5 0.0 0.001 1 2000
set 0 7 0.0 1.0 0.001 1 2000
set 4 8
hwstart
""")
if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode1-hwstart-test.png')

This produces the following scope trace:  
<img src="amp-test.png" alt="Amplitude Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is any of the 4 channel outputs from the AD9959

## Mode 2: Frequency Sweep

In [17]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 2: Frequency Sweep"')
send('reset')

'ok\n'

In [18]:
f112 = 112e6
f115 = 115e6
f118 = 118e6

d = 2000
t = 3000

send(
f"""abort
mode 2 1
setchannels 1
set 0 0 {f112} {f118} {d * 1} 1 {t * 1}
set 0 1 {f118} {f115} {d / 4} 1 {t * 2}
set 0 2 {f115} {f112} {d * 1} 1 {t / 2}
set 0 3 {f112} {f112} {d * 0} 1 {t * 1}
set 0 4 {f115} {f115} {d * 0} 1 {t * 1}
set 0 5 {f118} {f118} {d * 0} 1 {t * 1}
set 0 6 {f118} {f112} {d * 0} 1 {t * 1}
set 0 7 {f112} {f115} {d / 4} 1 {t * 2}
set 0 8 {f115} {f118} {d * 0} 1 {t * 10}
set 0 9 {f112} {f112} {d * 0} 1 1
set 4 10
start
"""
)

assert send('numtriggers') == '10\n'
print('Success')

if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode2-test.png')

Success


Produces the following scope trace:  
<img src="freq-test.png" alt="Frequency Sweep Example on oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is any of the 4 channel outputs from the AD9959  
The green trace is the output from an interferometer



The following code is for the same trace, but with an external trigger:

In [19]:
send(
f"""abort
mode 2 0
setchannels 1
set 0 0 {f112} {f118} {d} 1 {t}
set 0 1 {f118} {f115} {d} 1 {t}
set 0 2 {f115} {f112} {d} 1 {t}
set 0 3 {f112} {f115} {d} 1 {t}
set 0 4 {f115} {f118} {d} 1 {t}
set 0 5 {f118} {f112} {d} 1 {t}
set 5 6
hwstart
"""
)

'ok\nok\nok\n6\nok\n6\nok\n6\nok\n6\nok\n6\nok\n6\nok\n6\nok\nok\n'

## Mode 3: Phase Sweep

In [35]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 3: Phase Sweep"')
my_instrument.write(':TIMebase:DELay 20.0E-06')
my_instrument.write(':TIMebase:SCALe 20.0E-06')
send('reset')

'ok\n'

In [36]:
t = 2000
d = 0.2

send(f"""abort
debug off
setfreq 0 100000000
setfreq 1 100000000
setfreq 2 100000000
setfreq 3 100000000
setphase 0 0
setphase 1 0
setphase 2 0
setphase 3 0
mode 3 1
setchannels 2
set 0 0 0 0 0 0 {t}
set 0 1 0 0 0 0 {t}
set 0 2 0 0 0 0 {t}
set 0 3 0 0 0 0 {t}
set 0 4 0 0 0 0 {t}
set 0 5 0 0 0 0 {t}
set 1 0 0 180 {d} 1 {t}
set 1 1 180 90 {d} 1 {t}
set 1 2 90 0 {d} 1 {t}
set 1 3 0 90 {d} 1 {t}
set 1 4 90 180 {d} 1 {t}
set 1 5 180 0 {d} 1 {t}
set 4 6
start
""")

assert send('numtriggers') == '6\n'
print('Success')
if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode3-test.png')

Success


This produces the following scope trace:    
<img src="phase-test.png" alt="Phase Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is channel 1 from the AD9959  
The pink trace is the output of a phase frequency detector between channels 0 and 1 of the AD9959

## Mode 4: Amplitude Sweep and Single Step Freq, Phase

In [50]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 4: Amplitude Sweep, Single Step Freq, Phase"')
my_instrument.write(':TIMebase:DELay 35.0E-06')
my_instrument.write(':TIMebase:SCALe 10.0E-06')
send('reset')

'ok\n'

In [51]:
f2 = 125 * MHZ

d = 0.001
t = 2000


## set Chan Addr Start Stop Delta Rate Freq Phase Time
send(
f"""abort
mode 4 1
setchannels 2
set 0 0 1.0 0.0 {d * 1} 1 {f2} 0 {t}
set 0 1 0.0 0.5 {d * 1} 1 {f2} 0 {t}
set 0 2 0.5 1.0 {d * 1} 1 {f2} 0 {t}
set 0 3 1.0 0.0 {d * 1} 1 {f2} 0 {t}
set 0 4 0.0 1.0 {d * 1} 1 {f2} 0 {t}
set 0 5 1.0 0.5 {d * 1} 1 {f2} 0 {t}
set 0 6 0.5 0.0 {d * 1} 1 {f2} 0 {t}
set 0 7 0.0 1.0 {d * 1} 1 {f2} 0 {t}
set 0 8 1.0 1.0 {d * 0} 1 {f2} 0 {t}
set 0 9 1.0 1.0 {d * 0} 1 {f2} 0 {t}
# set 1 0 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 1 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 2 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 3 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 4 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 5 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 6 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 7 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 8 1.0 1.0 {d * 0} 1 {f2} 90 {t}
# set 1 9 1.0 1.0 {d * 0} 1 {f2} 90 {t}
set 4 10
start
"""
)

assert send('numtriggers') == '10\n'
print('Success')
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 4: Success"')

if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode4-test.png')

Success


## Mode 5: Frequency Sweep and Single Step Amp, Phase

In [24]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 5: Frequency Sweep, Single Step Amp, Phase"')


84

In [25]:
f112 = 112e6
f115 = 115e6
f118 = 118e6

d = 2000
t = 3000

## set Chan Addr Start Stop Delta Rate Amp Phase Time
send(
f"""abort
mode 5 1
setchannels 2
set 0 0 {f112} {f118} {d}   1 1 0 {t}
set 0 1 {f118} {f115} {d/4} 1 1 0 {t*2}
set 0 2 {f115} {f112} {d}   1 1 0 {t/2}
set 0 3 {f112} {f112} {0}   1 1 0 {t}
set 0 4 {f115} {f115} {0}   1 1 0 {t}
set 0 5 {f118} {f118} {0}   1 1 0 {t}
set 0 6 {f118} {f112} {d}   1 1 0 {t}
set 0 7 {f112} {f115} {d/4} 1 1 0 {t*2}
set 0 8 {f115} {f118} {d}   1 1 0 {t*10}
set 0 9 {f112} {f112}  0    1 1 0 1
set 1 0 {f112} {f118} {d}   1 0.75 90 {t}
set 1 1 {f118} {f115} {d/4} 1 0.5  180 {t*2}
set 1 2 {f115} {f112} {d}   1 0.9  90 {t/2}
set 1 3 {f112} {f112} {0}   1 1.0  00 {t}
set 1 4 {f115} {f115} {0}   1 0.9  90 {t}
set 1 5 {f118} {f118} {0}   1 0.8  180 {t}
set 1 6 {f118} {f112} {d}   1 0.9  90 {t}
set 1 7 {f112} {f115} {d/4} 1 1.0  00 {t*2}
set 1 8 {f115} {f118} {d}   1 0.9  90 {t*10}
set 1 9 {f112} {f112}  0    1 1.0  90 1
set 4 10
start
"""
)

assert send('numtriggers') == '10\n'
print('Success')

if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode5-test.png')

Success


This produces the following scope trace:    
<img src="all_sweep.png" alt="Phase Sweep Test on Oscilloscope">  
$D_1$ is the IO_UPDATE line between the pico and the AD9959.  
$D_0$ is the external trigger line into the pico.  
The yellow trace is channel 1 from the AD9959  
The pink trace is the output of a phase frequency detector between channels 0 and 1 of the AD9959
The green trace is the interferometer

## Mode 6: Phase Sweep and Single Step Amp, Freq

In [53]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing Mode 6: Phase Sweep, Single Step Amp, Freq"')
my_instrument.write(':TIMebase:DELay 20.0E-06')
my_instrument.write(':TIMebase:SCALe 20.0E-06')

26

In [54]:
f112 = 112e6
f115 = 115e6
f118 = 118e6

d = 0.2
t = 2000


### freq constant
## set Channel Address Start Stop Delta Rate Amp Freq Time
send(
f"""abort
mode 6 1
setchannels 2
set 0 0 0 90 {d} 1 1.00 {f112} {t * 2}
set 0 1 90 180 {d} 1 1.00 {f112} {t * 2}
set 0 2 180 270 {d} 1 1.00 {f112} {t * 2}
set 0 3 270 360 {d} 1 1.00 {f112} {t * 2}
set 0 4 360 270 {d} 1 1.00 {f112} {t * 2}
set 0 5 270 180 {d} 1 1.00 {f112} {t * 2}
set 0 6 180 270 {d} 1 1.00 {f112} {t * 2}
set 0 7 270 180 {d} 1 1.00 {f112} {t * 2}
set 0 8 180 90 {d} 1 1.00 {f112} {t * 2}
set 0 9 90 0 {d} 1 1.00 {f112} {t * 10}
set 1 0 0 0 90 {d} 1 1.00 {f112} {t * 2}
set 1 1 1 90 180 {d} 1 1.00 {f112} {t * 2}
set 1 2 2 180 270 {d} 1 1.00 {f112} {t * 2}
set 1 3 3 270 360 {d} 1 1.00 {f112} {t * 2}
set 1 4 4 360 270 {d} 1 1.00 {f112} {t * 2}
set 1 5 5 270 180 {d} 1 1.00 {f112} {t * 2}
set 1 6 6 180 270 {d} 1 1.00 {f112} {t * 2}
set 1 7 7 270 180 {d} 1 1.00 {f112} {t * 2}
set 1 8 8 180 90 {d} 1 1.00 {f112} {t * 2}
set 1 9 9 90 0 {d} 1 1.00 {f112} {t * 10}
set 4 10
start
"""
)

assert send('numtriggers') == '10\n'
print('Success')

if SAVE_IMAGES == True:
    my_instrument.save_screenshot('mode6-test.png')

Success


In [28]:
my_instrument.write(':DISPlay:ANNotation:TEXT "Testing finished"')

45